https://github.com/bradsun91/alpha-scientist/blob/master/content/05_Model_Scoring.ipynb

# Introduction

Use of machine learning in the quantitative investment field is, by all indications, skyrocketing. The proliferation of easily accessible data - both traditional and alternative - along with some very approachable frameworks for machine learning models - is encouraging many to explore the arena.

However, these financial ML explorers are learning that there are many ways in which using ML to predict financial time series differs greatly from labeling cat pictures or flagging spam. Among these differences is that traditional model performance metrics (RSQ, MSE, accuracy, F1, etc...) can be misleading and incomplete.

Over the past several years, I've developed a set of metrics which have proved useful for comparing and optimizing financial time series models. These metrics attempt to measure models' predictive power but also their trade-ability, critically important for those who actually intend to use their models in the real world.

In this post, I will present a general outline of my approach and will demonstrate a few of the most useful metrics I've added to my standard "scorecard". I look forward to hearing how others may think to extend the concept. If you'd like to replicate and experiment with the below code, you can download the source notebook for this post by right-clicking on the below button and choosing "save link as"

If you haven't already checked out the previous four installments in this tutorial, you may want review those first. Many of the coding patterns used below are discussed at length:

- Part 1: Data Management
- Part 2: Feature Engineering
- Part 3: Feature Selection
- Part 4: Walk-forward model building

# Preparing and Getting Started

In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline

In [6]:
location = "/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/data/" 
j = "j9000.csv"
jm = "jm000.csv"

In [7]:
j_df = pd.read_csv(location+j, engine="python", header=None)
jm_df = pd.read_csv(location+jm, engine='python', header=None)
j_df.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'holdings']
jm_df.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'holdings']

jm_df_cols = jm_df[['date', 'open', 'high', 'low', 'close', 'volume']]
j_df_cols = j_df[['date', 'open', 'high', 'low', 'close', 'volume']]

jm_df_cols['symbol'] = 'jm'
j_df_cols['symbol'] = 'j'

jm_df_cols['date'] = pd.to_datetime(jm_df_cols['date'])
j_df_cols['date'] = pd.to_datetime(j_df_cols['date'])

jm_df_cols = jm_df_cols.set_index(['date','symbol'])
j_df_cols = j_df_cols.set_index(['date', 'symbol'])

prices = pd.concat([jm_df_cols, j_df_cols]).sort_index()

In [8]:
prices.head()

open    high     low   close  volume
date                symbol                                        
2011-04-15 09:00:00 j       2275.0  2285.0  2250.0  2260.0   31834
2011-04-15 10:00:00 j       2259.0  2265.0  2252.0  2256.0    6440
2011-04-15 11:00:00 j       2256.0  2258.0  2228.0  2248.0    8264
2011-04-15 13:00:00 j       2246.0  2251.0  2238.0  2240.0    4832
2011-04-15 14:00:00 j       2241.0  2252.0  2239.0  2250.0    6710

The below code generates several features then synthetically generates an outcome series from them (along with noise). This guarantees that the features will be informative, since the outcome has been constructed to ensure a relationship.

In [9]:
num_obs = prices.close.count()

def add_memory(s,n_days=50,mem_strength=0.1):
    ''' adds autoregressive behavior to series of data'''
    add_ewm = lambda x: (1-mem_strength)*x + mem_strength*x.ewm(n_days).mean()
    out = s.groupby(level='symbol').apply(add_ewm)
    return out

# generate feature data
f01 = pd.Series(np.random.randn(num_obs),index=prices.index)
f01 = add_memory(f01,10,0.1)
f02 = pd.Series(np.random.randn(num_obs),index=prices.index)
f02 = add_memory(f02,10,0.1)
f03 = pd.Series(np.random.randn(num_obs),index=prices.index)
f03 = add_memory(f03,10,0.1)
f04 = pd.Series(np.random.randn(num_obs),index=prices.index)
f04 = f04 # no memory

features = pd.concat([f01,f02,f03,f04],axis=1)

## now, create response variable such that it is related to features
# f01 becomes increasingly important, f02 becomes decreasingly important,
# f03 oscillates in importance, f04 is stationary, 
# and finally a noise component is added

outcome =   f01 * np.linspace(0.5,1.5,num_obs) + \
            f02 * np.linspace(1.5,0.5,num_obs) + \
            f03 * pd.Series(np.sin(2*np.pi*np.linspace(0,1,num_obs)*2)+1,index=f03.index) + \
            f04 + \
            np.random.randn(num_obs) * 3 
outcome.name = 'outcome'

/Users/miaoyuesun/anaconda3/lib/python3.5/site-packages/pandas/compat/_optional.py:106: UserWarning: Pandas requires version '2.6.2' or newer of 'numexpr' (version '2.6.1' currently installed).
  warnings.warn(msg, UserWarning)


# Generating models and predictions

Imagine that we created a simple linear model (such as below) and wanted to measure its effectiveness at prediction.

Note: we'll follow the walk-forward modeling process described in the previous post. If you don't understand the below code snippet (and want to...) please check out that post.

In [26]:
from sklearn.linear_model import LinearRegression

## fit models for each timestep on a walk-forward basis
recalc_dates = features.resample('Q',level='date').mean().index.values[:-1]
models = pd.Series(index=recalc_dates)
for date in recalc_dates:
    X_train = features.xs(slice(None,date),level='date',drop_level=False)
    y_train = outcome.xs(slice(None,date),level='date',drop_level=False)
    model = LinearRegression()
    model.fit(X_train,y_train)
    models.loc[date] = model

## predict values walk-forward (all predictions out of sample)
begin_dates = models.index
end_dates = models.index[1:].append(pd.to_datetime(['2099-12-31']))

predictions = pd.Series(index=features.index)

for i,model in enumerate(models): #loop thru each models object in collection
    X = features.xs(slice(begin_dates[i],end_dates[i]),level='date',drop_level=False)
    p = pd.Series(model.predict(X),index=X.index)
    predictions.loc[X.index] = p

In [31]:
end_dates

DatetimeIndex(['2011-09-30', '2011-12-31', '2012-03-31', '2012-06-30',
               '2012-09-30', '2012-12-31', '2013-03-31', '2013-06-30',
               '2013-09-30', '2013-12-31', '2014-03-31', '2014-06-30',
               '2014-09-30', '2014-12-31', '2015-03-31', '2015-06-30',
               '2015-09-30', '2015-12-31', '2016-03-31', '2016-06-30',
               '2016-09-30', '2016-12-31', '2017-03-31', '2017-06-30',
               '2017-09-30', '2017-12-31', '2018-03-31', '2018-06-30',
               '2018-09-30', '2099-12-31'],
              dtype='datetime64[ns]', freq=None)

# Traditional model evaluation

So we've got a model, we've got a sizeable set of (out of sample) predictions. Is the model any good? Should we junk it, tune it, or trade it? Since this is a regression model, I'll throw our data into scikit-learn's metrics package.

In [34]:
import sklearn.metrics as metrics

# make sure we have 1-for-1 mapping between pred and true
# think of outcome as a fake series of stock prices - Brad
common_idx = outcome.dropna().index.intersection(predictions.dropna().index)

# y_true: just like fake stock prices - Brad
y_true = outcome[common_idx]
y_true.name = 'y_true'
# 
y_pred = predictions[common_idx]
y_pred.name = 'y_pred'

standard_metrics = pd.Series()

standard_metrics.loc['explained variance'] = metrics.explained_variance_score(y_true, y_pred)
standard_metrics.loc['MAE'] = metrics.mean_absolute_error(y_true, y_pred)
standard_metrics.loc['MSE'] = metrics.mean_squared_error(y_true, y_pred)
standard_metrics.loc['MedAE'] = metrics.median_absolute_error(y_true, y_pred)
standard_metrics.loc['RSQ'] = metrics.r2_score(y_true, y_pred)

print(standard_metrics)

explained variance    0.263251
MAE                   2.478018
MSE                   9.646065
MedAE                 2.098443
RSQ                   0.263251
dtype: float64


These stats don't really tell us much by themselves. You may have an intuition for r-squared so that may give you a level of confidence in the models. However, even this metric has problems not to mention does not tell us much about the practicality of this signal from a trading point of view.

True, we could construct some trading rules around this series of predictions and perform a formal backtest on that. However, that is quite time consuming and introduces a number of extraneous variables into the equation.

# A better way... Creating custom metrics

Instead of relying on generic ML metrics, we will create several custom metrics that will hopefully give a more complete picture of strength, reliability, and practicality of these models.

I'll work through an example of creating an extensible scorecard with about a half dozen custom-defined metrics as a starting point. You can feel free to extend this into a longer scorecard which is suited to your needs and beliefs. In my own trading, I use about 25 metrics in a standard "scorecard" each time I evaluate a model. You may prefer to use more, fewer, or different metrics but the process should be applicable.

I'll focus only on regression-oriented metrics (i.e., those which use a continuous prediction rather than a binary or classification prediction). It's trivial to re-purpose the same framework to a classification-oriented environment.

# Step 1: Preprocess data primitives

Before implementing specific metrics we need to do some data pre-processing. It'll become clear why doing this first will save considerable time later when calculating aggregate metrics.


To create these intermediate values, you'll need the following inputs:


- y_pred: the continuous variable prediction made by your model for each timestep, for each symbol
- y_true: the continuous variable actual outcome for each timestep, for each symbol.
- index: this is the unique identifier for each prediction or actual result. If working with a single instrument, then you can simply use date (or time or whatever). If you're using multiple instruments, a multi-index with (date/symbol) is necessary.


In other words, if your model is predicting one-day price changes, you'd want your y_pred to be the model's predictions made as of March 9th (for the coming day), indexed as 2017-03-09 and you'd want the actual future outcome which will play out in the next day also aligned to Mar 9th. This "peeking" convention is very useful for working with large sets of data across different time horizons. It is described ad nauseum in Part 1: Data Management.

The raw input data we need to provide might look something like this:

In [8]:
print(pd.concat([y_pred,y_true],axis=1).tail())

                     y_pred    y_true
date       symbol                    
2018-12-26 jm      0.210931  5.761904
2018-12-27 j      -0.497826 -5.743094
           jm     -0.613546  3.997778
2018-12-28 j      -0.546005  2.321911
           jm      1.128044 -2.139567


We will feed this data into a simple function which will return a dataframe with the y_pred and y_true values, along with several other useful derivative values. These derivative values include:

- sign_pred: positive or negative sign of prediction
- sign_true: positive or negative sign of true outcome
- is_correct: 1 if sign_pred == sign_true, else 0
- is_incorrect: opposite
- is_predicted: 1 if the model has made a valid prediction, 0 if not. This is important if models only emit predictions when they have a certain level of confidence
- result: the profit (loss) resulting from betting one unit in the direction of the sign_pred. This is the continuous variable result of following the model

In [42]:
def make_df(y_pred,y_true):
    y_pred.name = 'y_pred'
    y_true.name = 'y_true'
    
    df = pd.concat([y_pred,y_true],axis=1)

    df['sign_pred'] = df.y_pred.apply(np.sign)
    df['sign_true'] = df.y_true.apply(np.sign)
    df['is_correct'] = 0
    df.loc[df.sign_pred * df.sign_true > 0 ,'is_correct'] = 1 # only registers 1 when prediction was made AND it was correct
    df['is_incorrect'] = 0
    df.loc[df.sign_pred * df.sign_true < 0,'is_incorrect'] = 1 # only registers 1 when prediction was made AND it was wrong
    df['is_predicted'] = df.is_correct + df.is_incorrect
    df['result'] = df.sign_pred * df.y_true 
    return df

df = make_df(y_pred,y_true)
print(df.dropna().tail())

                              y_pred    y_true  sign_pred  sign_true  \
date                symbol                                             
2016-02-25 22:00:00 j      -1.381706 -2.701665       -1.0       -1.0   
2016-07-20 09:00:00 jm     -2.836265 -5.898601       -1.0       -1.0   
2011-07-01 11:00:00 j       0.056496 -1.300877        1.0       -1.0   
2015-11-03 22:00:00 jm      0.269227 -3.390068        1.0       -1.0   
2018-11-28 11:00:00 j       0.113279  2.230594        1.0        1.0   

                            is_correct  is_incorrect  is_predicted    result  
date                symbol                                                    
2016-02-25 22:00:00 j                1             0             1  2.701665  
2016-07-20 09:00:00 jm               1             0             1  5.898601  
2011-07-01 11:00:00 j                0             1             1 -1.300877  
2015-11-03 22:00:00 jm               0             1             1 -3.390068  
2018-11-28 11:00:00 j

In [43]:
df

y_pred    y_true  sign_pred  sign_true  \
date                symbol                                             
2017-09-15 22:00:00 jm      0.988718  3.391933        1.0        1.0   
2015-11-24 23:00:00 j      -1.790059  2.008035       -1.0        1.0   
2015-11-25 14:00:00 j      -3.325028 -1.058895       -1.0       -1.0   
2016-10-13 13:00:00 j       0.345094  2.259794        1.0        1.0   
2018-07-17 13:00:00 jm      1.359469  1.998052        1.0        1.0   
...                              ...       ...        ...        ...   
2016-02-25 22:00:00 j      -1.381706 -2.701665       -1.0       -1.0   
2016-07-20 09:00:00 jm     -2.836265 -5.898601       -1.0       -1.0   
2011-07-01 11:00:00 j       0.056496 -1.300877        1.0       -1.0   
2015-11-03 22:00:00 jm      0.269227 -3.390068        1.0       -1.0   
2018-11-28 11:00:00 j       0.113279  2.230594        1.0        1.0   

                            is_correct  is_incorrect  is_predicted    result  
date                symbol                                                    
2017-09-15 22:00:00 jm               1             0             1  3.391933  
2015-11-24 23:00:00 j                0             1             1 -2.008035  
2015-11-25 14:00:00 j                1             0             1  1.058895  
2016-10-13 13:00:00 j                1             0             1  2.259794  
2018-07-17 13:00:00 jm               1             0             1  1.998052  
...                                ...           ...           ...       ...  
2016-02-25 22:00:00 j                1             0             1  2.701665  
2016-07-20 09:00:00 jm               1             0             1  5.898601  
2011-07-01 11:00:00 j                0             1             1 -1.300877  
2015-11-03 22:00:00 jm               0             1             1 -3.390068  
2018-11-28 11:00:00 j                1             0             1  2.230594  

[22886 rows x 8 columns]

# Defining our metrics

With this set of intermediate variables pre-processed, we can more easily calculate metrics. The metrics we'll start with here include things like:

- Accuracy: Just as the name suggests, this measures the percent of predictions that were directionally correct vs. incorrect.

- Edge: perhaps the most useful of all metrics, this is the expected value of the prediction over a sufficiently large set of draws. Think of this like a blackjack card counter who knows the expected profit on each dollar bet when the odds are at a level of favorability

- Noise: critically important but often ignored, the noise metric estimates how dramatically the model's predictions vary from one day to the next. As you might imagine, a model which abruptly changes its mind every few days is much harder to follow (and much more expensive to trade) than one which is a bit more steady.

The below function takes in our pre-processed data primitives and returns a scorecard with accuracy, edge, and noise.

In [10]:
def calc_scorecard(df):
    scorecard = pd.Series()
    # building block metrics
    scorecard.loc['accuracy'] = df.is_correct.sum()*1. / (df.is_predicted.sum()*1.)*100
    scorecard.loc['edge'] = df.result.mean()
    scorecard.loc['noise'] = df.y_pred.diff().abs().mean()
    
    return scorecard    

calc_scorecard(df)

accuracy    66.089645
edge         1.450701
noise        2.280260
dtype: float64

Much better. I now know that we've been directionally correct about two-thirds of the time, and that following this signal would create an edge of ~1.5 units per time period.

Let's keep going. We can now easily combine and transform things to derive new metrics. The below function shows several examples, including:

- y_true_chg and y_pred_chg: The average magnitude of change (per period) in y_true and y_pred.
- prediction_calibration: A simple ratio of the magnitude of our predictions vs. magnitude of truth. This gives some indication of whether our model is properly tuned to the size of movement in addition to the direction of it.
- capture_ratio: Ratio of the "edge" we gain by following our predictions vs. the actual daily change. 100 would indicate that we were perfectly capturing the true movement of the target variable.

In [11]:
def calc_scorecard(df):
    scorecard = pd.Series()
    # building block metrics
    scorecard.loc['accuracy'] = df.is_correct.sum()*1. / (df.is_predicted.sum()*1.)*100
    scorecard.loc['edge'] = df.result.mean()
    scorecard.loc['noise'] = df.y_pred.diff().abs().mean()

    # derived metrics
    scorecard.loc['y_true_chg'] = df.y_true.abs().mean()
    scorecard.loc['y_pred_chg'] = df.y_pred.abs().mean()
    scorecard.loc['prediction_calibration'] = scorecard.loc['y_pred_chg']/scorecard.loc['y_true_chg']
    scorecard.loc['capture_ratio'] = scorecard.loc['edge']/scorecard.loc['y_true_chg']*100

    return scorecard    

calc_scorecard(df)

accuracy                  66.089645
edge                       1.450701
noise                      2.280260
y_true_chg                 2.860478
y_pred_chg                 1.617638
prediction_calibration     0.565513
capture_ratio             50.715321
dtype: float64

Additionally, metrics can be easily calculated for only long or short predictions (for a two-sided model) or separately for positions which ended up being winners and losers.

- edge_long and edge_short: The "edge" for only long signals or for short signals.
- edge_win and edge_lose: The "edge" for only winners or for only losers.

In [12]:
def calc_scorecard(df):
    scorecard = pd.Series()
    # building block metrics
    scorecard.loc['accuracy'] = df.is_correct.sum()*1. / (df.is_predicted.sum()*1.)*100
    scorecard.loc['edge'] = df.result.mean()
    scorecard.loc['noise'] = df.y_pred.diff().abs().mean()

    # derived metrics
    scorecard.loc['y_true_chg'] = df.y_true.abs().mean()
    scorecard.loc['y_pred_chg'] = df.y_pred.abs().mean()
    scorecard.loc['prediction_calibration'] = scorecard.loc['y_pred_chg']/scorecard.loc['y_true_chg']
    scorecard.loc['capture_ratio'] = scorecard.loc['edge']/scorecard.loc['y_true_chg']*100

    # metrics for a subset of predictions
    scorecard.loc['edge_long'] = df[df.sign_pred == 1].result.mean()  - df.y_true.mean()
    scorecard.loc['edge_short'] = df[df.sign_pred == -1].result.mean()  - df.y_true.mean()

    scorecard.loc['edge_win'] = df[df.is_correct == 1].result.mean()  - df.y_true.mean()
    scorecard.loc['edge_lose'] = df[df.is_incorrect == 1].result.mean()  - df.y_true.mean()

    return scorecard    

calc_scorecard(df)

accuracy                  66.089645
edge                       1.450701
noise                      2.280260
y_true_chg                 2.860478
y_pred_chg                 1.617638
prediction_calibration     0.565513
capture_ratio             50.715321
edge_long                  1.514464
edge_short                 1.530197
edge_win                   3.333592
edge_lose                 -2.006705
dtype: float64

From this slate of metrics, we've gained much more insight than we got from MSE, R-squared, etc...

- The model is predicting with a strong directional accuracy
- We are generating about 1.4 units of "edge" (expected profit) each prediction, which is about half of the total theoretical profit
- The model makes more on winners than it loses on losers
- The model is equally valid on both long and short predictions

If this were real data, I would be rushing to put this model into production!

# Metrics over time

Critically important when considering using a model in live trading is to understand (a) how consistent the model's performance has been, and (b) whether its current performance has degraded from its past. Markets have a way of discovering and eliminating past sources of edge.

Here, a two line function will calculate each metric by year:

In [13]:
def scorecard_by_year(df):
    df['year'] = df.index.get_level_values('date').year
    return df.groupby('year').apply(calc_scorecard).T

print(scorecard_by_year(df))

year                         2011       2012       2013       2014       2015  \
accuracy                74.603175  67.901235  66.197183  63.265306  65.368852   
edge                     2.099356   1.852989   1.568403   0.992050   1.406475   
noise                    2.547492   2.552125   2.571957   2.373444   2.231143   
y_true_chg               2.950821   2.995328   2.831913   2.762163   2.838464   
y_pred_chg               1.916061   1.849373   1.810558   1.681045   1.573468   
prediction_calibration   0.649332   0.617419   0.639341   0.608597   0.554338   
capture_ratio           71.144815  61.862645  55.383166  35.915705  49.550579   
edge_long                2.300789   1.935861   1.734747   1.000525   1.467651   
edge_short               2.272138   3.185282   1.190485   0.909155   1.562350   
edge_win                 3.570436   4.321410   3.193581   2.929456   3.357573   
edge_lose               -1.490579  -1.028126  -1.999003  -2.446903  -1.956827   

year                       

It's just as simple to compare performance across symbols (or symbol groups, if you've defined those):

In [14]:
def scorecard_by_symbol(df):
    return df.groupby(level='symbol').apply(calc_scorecard).T

print(scorecard_by_symbol(df))

symbol                          j         jm
accuracy                65.790914  66.477273
edge                     1.471711   1.423438
noise                    2.245444   2.239948
y_true_chg               2.825245   2.906197
y_pred_chg               1.632910   1.597822
prediction_calibration   0.577971   0.549798
capture_ratio           52.091462  48.979401
edge_long                1.475656   1.564949
edge_short               1.535812   1.527808
edge_win                 3.299720   3.377598
edge_lose               -1.944219  -2.090452


# Comparing models

The added insight we get from this methodology comes when wanting to make comparisons between models, periods, segments, etc...

To illustrate, let's say that we're comparing two models, a linear regression vs. a random forest, for performance on a training set and a testing set (pretend for a moment that we didn't adhere to Walk-forward model building practices...).

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNetCV,Lasso,Ridge
from sklearn.ensemble import RandomForestRegressor

X_train,X_test,y_train,y_test = train_test_split(features,outcome,test_size=0.20,shuffle=False)

# linear regression
model1 = LinearRegression().fit(X_train,y_train)
model1_train = pd.Series(model1.predict(X_train),index=X_train.index)
model1_test = pd.Series(model1.predict(X_test),index=X_test.index)

model2 = RandomForestRegressor().fit(X_train,y_train)
model2_train = pd.Series(model2.predict(X_train),index=X_train.index)
model2_test = pd.Series(model2.predict(X_test),index=X_test.index)

# create dataframes for each 
model1_train_df = make_df(model1_train,y_train)
model1_test_df = make_df(model1_test,y_test)
model2_train_df = make_df(model2_train,y_train)
model2_test_df = make_df(model2_test,y_test)

s1 = calc_scorecard(model1_train_df)
s1.name = 'model1_train'
s2 = calc_scorecard(model1_test_df)
s2.name = 'model1_test'
s3 = calc_scorecard(model2_train_df)
s3.name = 'model2_train'
s4 = calc_scorecard(model2_test_df)
s4.name = 'model2_test'

print(pd.concat([s1,s2,s3,s4],axis=1))

C:\Users\Brad Sun\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


                        model1_train  model1_test  model2_train  model2_test
accuracy                   66.945607    66.261398     88.817041    62.765957
edge                        1.515663     1.418232      2.670492     1.190708
noise                       2.078112     2.253423      3.085657     2.646271
y_true_chg                  2.870653     2.855090      2.870653     2.855090
y_pred_chg                  1.476714     1.582451      2.200979     1.894188
prediction_calibration      0.514417     0.554256      0.766717     0.663442
capture_ratio              52.798548    49.673816     93.027315    41.704741
edge_long                   1.544550     1.562990      2.673415     1.308844
edge_short                  1.706784     1.151250      2.890599     0.945018
edge_win                    3.387669     3.145336      3.231055     3.143665
edge_lose                  -1.937998    -2.208656     -0.783301    -2.314286


This quick and dirty scorecard comparison gives us a great deal of useful information. We learn that:

- The relatively simple linear regression (model1) does a very good job of prediction, correct about 68% of the time, capturing >50% of available price movement (this is very good) during training
- Model1 holds up very well out of sample, performing nearly as well on test as train
- Model2, a more complex random forest ensemble model, appears far superior on the training data, capturing 90%+ of available price action, but appears quite overfit and does not perform nearly as well on the test set.

# Summary

In this tutorial, we've covered a framework for evaluating models in a market prediction context and have demonstrated a few useful metrics. However, the approach can be extended much further to suit your needs. You can consider:

- Adding new metrics to the standard scorecard
- Comparing scorecard metrics for subsets of the universe. For instance, each symbol or grouping of symbols
- Calculating and plotting performance metrics across time to validate robustness or to identify trends

In the final post of this series, I'll present a unique framework for creating an ensemble model to blend together the results of your many different forecasting models.

Please feel free to add to the comment section with your good ideas for useful metrics, with questions/comments on this post, and topic ideas for future posts.

# One last thing...

If you've found this post useful, please follow @data2alpha on twitter and forward to a friend or colleague who may also find this topic interesting.

Finally, take a minute to leave a comment below - either to discuss this post or to offer an idea for future posts. Thanks for reading!